In [2]:
from google.colab import drive
drive.mount("/content/drive")
!pip install biopython

Mounted at /content/drive


In [2]:
!pip install biopython

In [26]:

from Bio import Entrez, SeqIO
from datetime import datetime
from multiprocessing import Process
import multiprocessing
import pandas , csv
#CSVs = ["Ino_Prot.csv","Myo_Prot.csv","Podo_Prot.csv","Sipho_Prot.csv"]
#SeqCSVs = ["Ino_Prot_Seq.csv","Myo_Prot_Seq.csv","Podo_Prot_Seq.csv","Sipho_Prot_Seq.csv"]
CSVs = ["Myo_Prot.csv","Podo_Prot.csv","Sipho_Prot.csv"]
SeqCSVs = ["Myo_Prot_Seq.csv","Podo_Prot_Seq.csv","Sipho_Prot_Seq.csv"]
dataL = []

def readCSV():
  for csv in CSVs:
    try:
      data = pandas.read_csv("/content/drive/My Drive/FYP/"+csv)
    except Exception as e:
      data = pandas.read_csv(csv)
    dataL.append(data)

def getProtSeqByID(id:str , index:int,seqDescriptionDict:dict):
  print("!1")
  count = 0
  while count < 4:
    try:
      Entrez.email = "cyli77-c@my.cityu.edu.hk"  
      handle = Entrez.esearch(db="protein", term=id)
      read_gene = Entrez.read(handle)
      #print(read_gene['IdList'])
      handle = Entrez.efetch(db="protein", id=read_gene['IdList'], rettype="fasta", retmode="text")
      #print (handle.read())
      seq_record = SeqIO.read(handle, "fasta")
      #print (seq_record.seq)
      seqDescriptionDict[index] = (seq_record.seq,seq_record.description)
      return True
    except Exception as e:
      #Try agin if any error
      count += 1
      print ("{} is trying to fetch again".format(protId))
    return False

def writeSeq2CSV():
  for seqcsv , data in zip(SeqCSVs,dataL):
    with open(seqcsv,"w",newline="") as csvfile:
      writer = csv.writer(csvfile)
      writer.writerow(["prot_tag", "prot_id", "name","size","seq","description"])
      finishedCount = 0
      #Start loop
      PROCESS_NUM = 5
      while finishedCount != len(data):
        remainCount = len(data)-finishedCount
        processCount = 0
        # For create process
        if remainCount>PROCESS_NUM:
          processCount = PROCESS_NUM
        else:
          processCount = remainCount
        #each time processCount
        manager = multiprocessing.Manager()
        seqDescriptionDict =  manager.dict()
        protTags = []
        protIds = []
        protNames = []
        protLengths = []
        processL = []
        for i in range(processCount):
          protTags.append(data["prot_tag"][finishedCount+i])
          protIds.append(data["prot_id"][finishedCount+i])
          protNames.append(data["name"][finishedCount+i])
          protLengths.append(data["size"][finishedCount+i])
          now = datetime.now()
          curTime = now.strftime("%H:%M:%S")
          print ("{} {} Doing index:{} {}".format(curTime ,seqcsv,i,protIds[i]))
          tuplesID = tuple(protIds)
          p = Process(target = getProtSeqByID,args=(tuplesID[i],i,seqDescriptionDict))
          processL.append(p)
          p.start()
          finishedCount += 1
          

        for i in range(len(processL)):
          processL[i].join()
          print(seqDescriptionDict)
          seq = seqDescriptionDict[i][0]
          des = seqDescriptionDict[i][1]
          writer.writerow([protTags[i],protIds[i],protNames[i],protLengths[i],seq,des])
      csvfile.truncate()
      csvfile.close()
readCSV()
writeSeq2CSV()

16:59:43 Myo_Prot_Seq.csv Doing index:0 NP_899486.1
16:59:43 Myo_Prot_Seq.csv Doing index:1 YP_009201108.1
16:59:43 Myo_Prot_Seq.csv Doing index:2 AFN37469.1
16:59:43 Myo_Prot_Seq.csv Doing index:3 ALP47132.1
16:59:43 Myo_Prot_Seq.csv Doing index:4 ALP47516.1
{}


KeyError: 0

In [ ]:
!pip install biopython
from Bio import Entrez
hd_info = Entrez.einfo()
# get the database onfo
read_info = Entrez.read(hd_info)
"""
for db in read_info['DbList']:
    print (db)
"""

ModuleNotFoundError: ignored

# 新增區段

MANILYGSDAVAQVEAAEGPLSSTERRIVELEGLADGTYADTKGIATSGVGQTGQFANMSFKDTVATFEEETRRMIPNFDSLSPKLQTELVQAAYRGDLQQSPKTRALINEGNFEGAAQEFLNNQEFQTTPYNAIRDRMADVSTALAAEGAQAPVAPSVGEQKQVQLANRTPQKQQQVANTSDSKQLELAGLSQAEQRILDLGMSEYVRNGQTGTAASRAQEYMDAQDKLSTIQQRQDEESTKVIFEDLDNPLQATGNTLLSVANTIGNIAGGLVSLPSSIRSAYNLSDATNEGFAQYQSILAKEQAGVALTPAEREFQGTDEYSDIKQGMESMARAQSIQGGFGILGDYVNQELTTEFVNEGSKLFNEVAGGDITMGEFISESARLLTQSPGAIGQATVESLPYVLASAYNLPLTVASGWANNQQEALEEYQKNNDGAIPTGRDLANVTMLAGGQALVDHFGDRLVGGKNLKNLISGATGRRSAGAAVGVVQGTAGEFVAEGGAEVLGQLAATAGKGEVDLAQAYAAGLLGAGAGGATSGAIAGANITADAIDNAPDLGEKVSSVSERAKGAVESVKTAATEAKDAATKAAQTTTEEQTEAPTEEKGLDVRLSEAFVESDSNPDQAQTIEADVRAEYDALKTEAQDIVKSGVTNETITEEQKDTIIRFQEAQKVIRARNKEVKAREAAKQANQLATASTNNTATTTAVENVLGSIELDPSSISADQAEAILGNQNLRLDAEQRSKLQHHVRVAKTLDEVSNEIRDGGNGNIGINQYIEEVGAALEIGDVATATESLDALKAWQQQHVAKLQSNVHNGKPMTPRLRAQVAKEVKAMNAAVAKLENDLIYADAPTQEEQANVDANVPTTDNVGGTTTDDTEVRAEQQAPQAVAEDAPAAEANQEQVTAELREIFQELDSDPALVDQVVTGVLDLNEQGVTFDSALTSLKEQYGENTIKAAEESLAKLFPTTRESLAGRMTTPEGGGLKIPVMPEGLTAKEK